# Frontier Network Maintenance Detector
## By Mark Friant and Keshav Santhanam




### Imports and Data

In [209]:
import numpy as np, pandas as pd, torch, keras
import tensorflow as tf
import re
import math
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [210]:
train_data = pd.read_csv("https://raw.githubusercontent.com/KeshavSanthanam/Hackathon2023/main/Data_Challenge/Data/training_data.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/KeshavSanthanam/Hackathon2023/main/Data_Challenge/Data/test_data.csv")

<ipython-input-210-75d6effb9bb1>:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv("https://raw.githubusercontent.com/KeshavSanthanam/Hackathon2023/main/Data_Challenge/Data/training_data.csv")


### Populating service_repair:
Empty "cells" in a CSV file are considered as null in our dataframes, but that isn't a descriptive way to represent the lack of a repair when a repair is indicated by 1. We substitute the nulls with 0's to better represent the data, but this doesn't change our results.

In [211]:
# replace service_repair nulls with 0 (since no service repair occured)
train_data['service_repair'].fillna(0, inplace=True)

### Dropping REPORT_DATE
REPORT_DATE is, by definition, a one-to-one match with each occurence of service_repair, and it obviously is not recorded in test_data, so it is essential that we remove it since it contains the "answer" to our model. We also drop 5 empty columns to limit the amount of processing power needed to develop our model.

In [212]:
# remove REPORT_DATE
train_data = train_data.drop('REPORT_DATE', axis=1)
train_data = train_data.drop('Stops_null', axis=1)
train_data = train_data.drop('Stops_NAS-Error', axis=1)
train_data = train_data.drop('Stops_Port-Error', axis=1)
train_data = train_data.drop('Stops_Service-Unavailable', axis=1)
train_data = train_data.drop('Stops_User-Error', axis=1)

test_data = test_data.drop('Stops_null', axis=1)
test_data = test_data.drop('Stops_NAS-Error', axis=1)
test_data = test_data.drop('Stops_Port-Error', axis=1)
test_data = test_data.drop('Stops_Service-Unavailable', axis=1)
test_data = test_data.drop('Stops_User-Error', axis=1)

for col in test_data.columns:
  if test_data[col].isnull().any():
    test_data.drop(columns=col, inplace=True)
    train_data.drop(columns=col, inplace=True)

### Our Sequential Neural Network
We hash our strings to integers in order to use tensors and then construct a feedforward neural network with 60 nodes on input (to match the attributes) and a single output node to match the binary output.

In [213]:
train_feature_data = train_data.drop('service_repair', axis=1)
train_label_data = train_data['service_repair']

def sub_value(x):
  if type(x) == str:
    return 0 if x == '' else hash(x)
  else:
    return 0 if math.isnan(x) else x

def my_hash(df):
    for column in df.columns:
      df[column] = df[column].apply(lambda x: sub_value(x))
    return df

train_feature_data = my_hash(train_feature_data)

In [214]:
train_input, test_input, train_output, test_output = train_test_split(train_feature_data, train_label_data, test_size=0.2, random_state=42, shuffle=True)
test_input, test_input2, test_output, test_output2 = train_test_split(test_input, test_output, test_size=0.5, random_state=42, shuffle=True)
errors, accuracies = [], []
errors2, accuracies2 = [], []

model = Sequential()
model.add(Dense(60, input_dim=train_feature_data.shape[1], activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=1)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 60)                1440      
                                                                 
 dense_40 (Dense)            (None, 12)                732       
                                                                 
 dense_41 (Dense)            (None, 1)                 13        
                                                                 
Total params: 2185 (8.54 KB)
Trainable params: 2185 (8.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [215]:
history = model.fit(tf.convert_to_tensor(train_input.to_numpy()), tf.convert_to_tensor(train_output.to_numpy()), epochs=10, batch_size=32, initial_epoch=0, validation_data=(test_input, test_output), callbacks=[early_stopping])

Epoch 1/10
1577/1577 [==============================] - 44s 28ms/step - loss: 7342599316701184.0000 - accuracy: 0.9589 - val_loss: 5889467462189056.0000 - val_accuracy: 0.9613
Epoch 2/10
1577/1577 [==============================] - 45s 29ms/step - loss: 3407641279725568.0000 - accuracy: 0.9582 - val_loss: 3896734438653952.0000 - val_accuracy: 0.9667
Epoch 3/10
1577/1577 [==============================] - 44s 28ms/step - loss: 2292039899152384.0000 - accuracy: 0.9583 - val_loss: 3236356641783808.0000 - val_accuracy: 0.9735
Epoch 4/10
1577/1577 [==============================] - 45s 28ms/step - loss: 1629023381749760.0000 - accuracy: 0.9585 - val_loss: 2130826691084288.0000 - val_accuracy: 0.9711


In [216]:
scores = model.evaluate(test_input, test_output)
errors.append(scores[0])
accuracies.append(scores[1])
print(accuracies)

198/198 [==============================] - 2s 12ms/step - loss: 2130826691084288.0000 - accuracy: 0.9711
[0.9711431860923767]


In [217]:
scores = model.evaluate(test_input2, test_output2)
errors2.append(scores[0])
accuracies2.append(scores[1])
print(accuracies2)

198/198 [==============================] - 3s 12ms/step - loss: 1787338963288064.0000 - accuracy: 0.9745
[0.9744768738746643]


In [219]:
# prediction = model.predict(tf.convert_to_tensor(test_input.to_numpy()))

test_data = my_hash(test_data)
mp = model.predict(test_data)
out_df = pd.DataFrame(mp)
out_df.insert(0, "Customer", test_data['Customer'])
out_df = out_df.rename(columns={out_df.columns[1]: 'Service_Repair'})
out_df.to_csv('output.csv', index=False)

# type(test_data['TSO_calls_count'][5])
# math.isnan(test_data['TSO_calls_count'][5])
#model.predict(test_data)
  # print(model.predict(row.to_numpy()))

# prediction = model.predict(test_input.to_numpy())

# out_file = "output.csv"
# out_df[0] = test_input.iloc[0]
# out_df[1] = prediction
# out_df = pd.DataFrame(out_np)

# out_df.to_csv(out_file, sep=',')

1911/1911 [==============================] - 8s 4ms/step
